In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install ultralytics opencv-python-headless

import torch
import cv2
import os
from google.colab import files

from ultralytics import YOLO

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.7/793.7 kB 10.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [ ]:
model = YOLO('/content/gdrive/MyDrive/Yolo v9c 100 Epoch train/weights/best.pt')

In [ ]:
uploaded = files.upload()
video_path = ''

# Move the uploaded video to a specific directory
os.makedirs('videos', exist_ok=True)
for file_name in uploaded.keys():
    video_path = 'videos/demo' + file_name.split('.')[1]
    os.rename(file_name, os.path.join(video_path))

Saving 3.mp4 to 3.mp4


In [ ]:
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform detection
    results = model(frame)

    # Draw bounding boxes on the frame
    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            conf = box.conf[0].cpu().numpy()
            cls = int(box.cls[0].cpu().numpy())
            label = f'{model.names[cls]} {conf:.2f}'
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Write the frame into the output file
    out.write(frame)

    # Optionally display the frame (comment out if not needed)
    #cv2.imshow('Frame', frame)
    #if cv2.waitKey(1) & 0xFF == ord('q'):
    #  break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

from google.colab import files
files.download('output.avi')


0: 384x640 1 Batsman, 1 Bowler, 1 Fielder, 1 Umpire, 1 Non-strike-Batsman, 2243.7ms
Speed: 17.6ms preprocess, 2243.7ms inference, 32.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Batsman, 1 Bowler, 1 Fielder, 1 Umpire, 1 Non-strike-Batsman, 1491.3ms
Speed: 5.4ms preprocess, 1491.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Batsman, 1 Bowler, 1 Fielder, 1 Umpire, 1 Non-strike-Batsman, 1251.1ms
Speed: 2.7ms preprocess, 1251.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Batsman, 1 Bowler, 1 Umpire, 1 Keeper, 1 Non-strike-Batsman, 1297.0ms
Speed: 2.8ms preprocess, 1297.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Batsman, 1 Bowler, 1 Umpire, 1 Keeper, 1 Non-strike-Batsman, 1250.2ms
Speed: 2.4ms preprocess, 1250.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Batsman, 1 Bowler, 1 Umpire, 1 Keeper, 1 Non-strike-Batsman, 1631.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#The following code is for cropping and stretching
import numpy as np

cap = cv2.VideoCapture(video_path)

output_size = (256, 256)
out = cv2.VideoWriter('output2.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 30, output_size)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform detection
    results = model(frame)

    # Flag to check if "Batsman" is detected
    batsman_detected = False

    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            cls = int(box.cls[0].cpu().numpy())
            label = model.names[cls]

            if label == 'Batsman':  # Ensure the class name matches
                # Crop the frame to the bounding box
                cropped_frame = frame[y1:y2, x1:x2]

                # Resize the cropped frame to 256x256
                resized_frame = cv2.resize(cropped_frame, output_size)

                # Write the resized frame into the output file
                out.write(resized_frame)

                # Set the flag
                batsman_detected = True
                break

        if batsman_detected:
            break
        else:
          continue

    # Optionally display the frame (comment out if not needed)
    # cv2.imshow('Frame', resized_frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

# Download the output file
from google.colab import files
files.download('output2.avi')


0: 384x640 1 Batsman, 1 Bowler, 2 Fielders, 1 Umpire, 1 Keeper, 1 Non-strike-Batsman, 2336.3ms
Speed: 18.1ms preprocess, 2336.3ms inference, 28.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Batsman, 1 Bowler, 2 Fielders, 1 Umpire, 1 Keeper, 1 Non-strike-Batsman, 2313.2ms
Speed: 2.7ms preprocess, 2313.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Batsman, 1 Bowler, 2 Fielders, 1 Umpire, 1 Keeper, 1 Non-strike-Batsman, 2249.9ms
Speed: 6.4ms preprocess, 2249.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Batsman, 1 Bowler, 2 Fielders, 1 Umpire, 1 Keeper, 1 Non-strike-Batsman, 2292.0ms
Speed: 3.5ms preprocess, 2292.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Batsman, 1 Bowler, 2 Fielders, 1 Umpire, 1 Keeper, 1 Non-strike-Batsman, 1428.9ms
Speed: 4.2ms preprocess, 1428.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Batsman, 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>